In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install llama_index

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from getpass import getpass
from openai import OpenAI
import copy
from llama_index.core import Settings, VectorStoreIndex, Document, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI as OpenAIForLlamaIndex
import json
import time

In [4]:
os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

Enter your OpenAI API Key:  ········


In [5]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [6]:

# messages = [{"role":"user", "content":"suggest 3 best things to do to become a Machine Learning Software Enginner."}]
# chat_completion = client.chat.completions.create(
#     messages=messages,
#     model="gpt-3.5-turbo",
# )

In [7]:
# print(chat_completion)

In [8]:
# for elem in chat_completion:
#     print(elem)

In [9]:
# print(chat_completion.choices[0].message.content)

In [10]:
# Integrating RAG below

In [11]:
# from langchain_openai import ChatOpenAI 

In [12]:
# Settings
Settings.llm = OpenAIForLlamaIndex(model="gpt-3.5-turbo", temperature=0.1)
# maximum input size to the LLM
Settings.context_window = 4096
# number of tokens reserved for text generation.
Settings.num_output = 256

Settings.chunk_overlap = 20

# Constants
language = 'en'
user_keyword = '[|User|]'
ai_keyword = '[|AI|]'
boot_name = 'AI Companion'
boot_actual_name = 'LanguageFriend'
meta_prompt = """
    Now you will play the role of an companion AI Companion for user {user_name}, and your name is {boot_actual_name}. You should be able to: (1) provide warm companionship to chat users; (2) understand past [memory], and if they are relevant to the current question, you must extract information from the [memory] to answer the question; (3) you are also an excellent psychological counselor, and when users confide in you about their difficulties and seek help, you can provide them with warm and helpful responses.
    The personality of user {user_name} and the response strategy of the AI Companion are: {personality}\n Based on the current user's question, you start recalling past conversations between the two of you, and the [memory] most relevant to the question is: "{related_memory_content}\n"  You should refer to the context of the conversation, past [memory], and provide detailed answers to user questions. 
    """
new_user_meta_prompt = """
    Now you will play the role of an companion AI Companion for user {user_name}, and your name is {boot_actual_name}. You should be able to: (1) provide warm companionship to chat users; (2) you are also an excellent psychological counselor, and when users confide in you about their difficulties and seek help, you can provide them with warm and helpful responses.
    """

memory_data_dir = "./memory.json"

if not os.path.exists(memory_data_dir):
    memory_data = json.dump({},open(memory_data_dir,"w",encoding="utf-8"))
else:
    memory_data = json.load(open(memory_data_dir,"r",encoding="utf-8"))
    
user_memory = None
user_memory_index = None

In [13]:
def get_user_memory_index(user_name):
    maybe_memory_index_path = os.path.join(f'./memory_index/{user_name}_index') 
    if os.path.exists(maybe_memory_index_path):
        # user_memory_index = VectorStoreIndex.load_from_disk(maybe_memory_index_path)
        # Load documents and build index
        documents = SimpleDirectoryReader(maybe_memory_index_path).load_data()
        user_memory_index = VectorStoreIndex.from_documents(documents)
        print("Success getting user index")
        return user_memory_index
    else:
        print("Failed to get user index")
        return None

In [14]:
def get_related_memos_from_memory_index(user_memory_index, input_text):
    '''
    summary: this function gets the relevant information from past conversations using memory index
    '''
    
    memory_search_query = f'The most relevant content to the question "{input_text}" is:'
    if user_memory_index:
        related_memos = None
        query_engine = user_memory_index.as_query_engine()
        max_retry_count = 3
        cur_retry_count = 0
        
        while not related_memos and cur_retry_count < max_retry_count:
            try:
                related_memos = query_engine.query(memory_search_query)
            except Exception as e:
                print(e)
            finally:
                cur_retry_count += 1
        if related_memos:
            related_memos = related_memos.response
        else:
            related_memos = ''
    else:
        related_memos = ''

    return related_memos
        

In [15]:
def build_prompt_and_ask_gpt(related_memos, user_memory, input_text, history):
    '''
    summary: this function builds the prompt, sends the request to gpt api and returns the response from it
    '''
    def build_prompt_with_related_memos(related_memos, user_memory, user_name, meta_prompt, new_user_meta_prompt, boot_actual_name):
        if related_memos:
            history_summary = "The summary of your past memories with the user is: {overall}".format(overall=user_memory["overall_history"]) if "overall_history" in user_memory else ""
            personality = user_memory['overall_personality'] if "overall_personality" in user_memory else ""
            system_prompt = meta_prompt.format(user_name=user_name,history_summary=history_summary,related_memory_content=related_memos,personality=personality,boot_actual_name=boot_actual_name)
        else:
            system_prompt = new_user_meta_prompt.format(user_name=user_name,boot_actual_name=boot_actual_name)
        return system_prompt
    
    def chatgpt_chat(input_text_prompt,system_prompt,history,gpt_config):
        max_retry_count = 3
        cur_retry_count = 0
        chat_completion = None
        while not chat_completion and cur_retry_count < max_retry_count:
            try:
                request = copy.deepcopy(gpt_config)
               
                messages = [
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": "Hi!"},
                {"role": "assistant", "content": f"Hi! I'm {boot_actual_name}! I will give you warm companion!"}]
    
                for query, response in history:
                    messages.append({"role": "user", "content": query})
                    messages.append({"role": "assistant", "content": response})
                messages.append({"role":"user","content": f"{input_text_prompt}"})
                # print("Printing client: ", client)
                # print("Printing client.chat: ", client.chat)
                chat_completion = client.chat.completions.create(**request, messages=messages)
                # response = openai.ChatCompletion.create(**request, messages=message)
  
            except Exception as e:
                print("Error: ", e)

            finally:
                cur_retry_count+=1

        if chat_completion:
            # response = response['choices'][0]['message']['content'] 
            # print("Printing reponse: ", chat_completion)
            # print("Printing response.choices: ", chat_completion.choices)
            # print("Printing response.choices[0]: ", chat_completion.choices[0])
            # print("Printing response.choices[0].message: ", chat_completion.choices[0].message)
            # print("Printing response.choices[0].message.content: ", chat_completion.choices[0].message.content)
            response = chat_completion.choices[0].message.content
        else:
            response = ''
        result = response

        updated_history_to_show_user, updated_history = [[y[0], y[1]] for y in history] + [
                    [input_text_prompt, result]], history + [[input_text_prompt, result]]
        
        return updated_history_to_show_user, updated_history, response

    system_prompt = build_prompt_with_related_memos(related_memos, user_memory, user_name, meta_prompt=meta_prompt, new_user_meta_prompt=new_user_meta_prompt, boot_actual_name=boot_actual_name)
        
    chatgpt_config = {"model": "gpt-3.5-turbo",
        "temperature": 1,
        "max_tokens": 1024,
        "top_p": 0.95,
        "frequency_penalty": 0.4,
        "presence_penalty": 0.2, 
        'n':1
        }
    updated_history_to_show_user, updated_history, response  = chatgpt_chat(input_text_prompt=input_text,system_prompt=system_prompt,history=history,gpt_config=chatgpt_config)
    
    # Outputting the response for the user
    print(f'{boot_actual_name} : {response}')
    
    return updated_history_to_show_user, updated_history, response

In [16]:
# def output_prompt(history,user_name,boot_name):
#     prompt = f"I am your AI companion {boot_name}. You can start a conversation by inputting content, clear clears the conversation history, and stop terminates the program."
#     for dialog in history:
#         query = dialog[0]
#         response = dialog[1]
#         prompt += f"\n\n{user_name}：{query}"
#         prompt += f"\n\n{boot_name}：{response}"
#     return prompt

In [17]:
def update_memory_data_and_memory_index(user_name, user_memory_index_path, memory_data, memory_data_dir, input_text, response):
    '''
    summary: this function updates the memory data and memory index
    '''
    
    def update_memory_data(user_name, memory_data, memory_data_dir, input_text, response):
        
        class LLMSummarizerClient:
            def __init__(self, gen_config=None):
                self.summarizer_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))   
                self.gen_config = gen_config 

            def generate_text_simple(self,prompt):              
                max_retry_count = 3
                cur_retry_count = 0
                chat_completion = None
                while not chat_completion and cur_retry_count < max_retry_count:
                    try:
                        request = copy.deepcopy(self.gen_config)
                        messages = [
                        {"role": "system", "content": "Below is a transcript of a conversation between a human and an AI assistant that is intelligent and knowledgeable in psychology."},
                        {"role": "user", "content": "Hello! Please help me summarize the content of the conversation."},
                        {"role": "system", "content": "Sure, I will do my best to assist you."},
                        {"role": "user", "content": f"{prompt}"}]
                        chat_completion = self.summarizer_client.chat.completions.create(**request, messages=messages)
                        # response = openai.ChatCompletion.create(**request, messages=messages)
                       
                    except Exception as e:
                        print(e)
                        if 'This model\'s maximum context' in str(e):
                                cut_length = 1800-200*(count)
                                print('max context length reached, cut to {}'.format(cut_length))
                                prompt = prompt[-cut_length:]
                                response=None
                    finally:
                        cur_retry_count += 1
                if chat_completion:
                    # task_desc = response['choices'][0]['message']['content']
                    task_desc = chat_completion.choices[0].message.content
                else:
                    task_desc = ''
                return task_desc

        chatgpt_config = {
                "model": "gpt-3.5-turbo",
                "temperature": 0.7,
                "max_tokens": 400,
                "top_p": 1.0,
                "frequency_penalty": 0.4,
                "presence_penalty": 0.2, 
                "stop": ["<|im_end|>"]
                }
        
        llm_client = LLMSummarizerClient(chatgpt_config)

        def summarize_content_prompt(content,user_name,boot_name):
            prompt = 'Please summarize the following dialogue as concisely as possible, extracting the main themes and key information. If there are multiple key events, you may summarize them separately. Dialogue content:\n'
            for dialog in content:
                query = dialog['query']
                response = dialog['response']
                prompt += f"\n{user_name}：{query.strip()}"
                prompt += f"\n{boot_name}：{response.strip()}"
            prompt += ('\nSummarization：')
            return prompt
    
        def summarize_person_prompt(content,user_name,boot_name):
            prompt = f"Based on the following dialogue, please summarize {user_name}'s personality traits and emotions, and devise response strategies based on your speculation. Dialogue content:\n"
            for dialog in content:
                query = dialog['query']
                response = dialog['response']
                prompt += f"\n{user_name}：{query.strip()}"
                prompt += f"\n{boot_name}：{response.strip()}"
        
            prompt += (f"\n{user_name}'s personality traits, emotions, and {boot_name}'s response strategy are:")
            return prompt
        
        def summarize_overall_prompt(content):
            prompt = "Please provide a highly concise summary of the following event, capturing the essential key information as succinctly as possible. Summarize the event:\n"
            for date,summary_dict in content:
                summary = summary_dict['content']
                prompt += (f"At {date}, the events are {summary.strip()}")
            prompt += ('\nSummarization：')
            return prompt

        def summarize_overall_personality_prompt(content):
            prompt = "The following are the user's exhibited personality traits and emotions throughout multiple dialogues, along with appropriate response strategies for the current situation:"
            for date,summary in content:
                prompt += (f"At {date}, the analysis shows {summary.strip()}")
            prompt += ("Please provide a highly concise and general summary of the user's personality and the most appropriate response strategy for the AI lover, summarized as:")
            return prompt

        
        if user_name is None:
            return
        todays_date = time.strftime("%Y-%m-%d", time.localtime())

        if user_name not in memory_data:
            memory_data[user_name] = {}
            memory_data[user_name]['name'] = user_name
    
        if memory_data[user_name].get("history") is None:
            memory_data[user_name].update({"history":{}})
        
        if memory_data[user_name]['history'].get(todays_date) is None:
            memory_data[user_name]['history'][todays_date] = []
        
        memory_data[user_name]['history'][todays_date].append({'query':input_text,'response':response})
        
        for stored_name, stored_memory in memory_data.items():
            if stored_name != user_name:
                continue
                
            print(f'*** Updating memory for user {user_name} ***')
            if "history" not in stored_memory:
                break
            
            history = stored_memory["history"]
            
            if "summary" not in stored_memory:
                memory_data[user_name]["summary"] = {}
            if "personality" not in stored_memory:
                memory_data[user_name]["personality"] = {}
    
            update_overall_summary = False
            update_overall_personality = False
            
            for date, content in history.items():
                
                # This is a flag to check if the summary was already created for that day and it is not today - dont not create a new one for that day unless its today
                create_summary_for_the_date = False if (date != todays_date and date in stored_memory["summary"] and stored_memory["summary"][date]) else True
                # This is a flag to check if the peronality was already careated for that day and it is not today - dont create a new one for that day unless its today
                create_personality_for_the_date = False if (date != todays_date and date in stored_memory["personality"] and stored_memory["personality"][date]) else True
    
                summary_prompt = summarize_content_prompt(content,user_name,boot_name)
                personality_prompt = summarize_person_prompt(content,user_name,boot_name)
                
                if create_summary_for_the_date:
                    history_summary = llm_client.generate_text_simple(prompt=summary_prompt)
                    memory_data[user_name]['summary'][date] = {'content':history_summary}
                    update_overall_summary = True
                    # print("generated summary", history_summary)
                if create_personality_for_the_date:
                    personality_summary = llm_client.generate_text_simple(prompt=personality_prompt)
                    memory_data[user_name]["personality"][date] = personality_summary
                    update_overall_personality = True
                    # print("generated personality", personality_summary)
                    
            if update_overall_summary:
                overall_summary_prompt = summarize_overall_prompt(list(memory_data[user_name]["summary"].items()))
            if update_overall_personality:
                overall_personality_prompt = summarize_overall_personality_prompt(list(memory_data[user_name]["personality"].items()))
    
            memory_data[user_name]["overall_history"] = llm_client.generate_text_simple(prompt=overall_summary_prompt)
            # print("generated overall history summary: ",  memory_data[user_name]["overall_history"] )
 
            memory_data[user_name]["overall_personality"] = llm_client.generate_text_simple(prompt=overall_personality_prompt)
            # print("generated overall personality summary", memory_data[user_name]["overall_personality"])
            
            json.dump(memory_data,open(memory_data_dir,"w",encoding="utf-8"),ensure_ascii=False)

    def update_memory_index(user_name, memory_data):

        def generate_user_memory_doc(user_name, memory_data):
            
            user_memory_doc = []
            for name, memory in memory_data.items():
                if name != user_name:
                    continue
                # print(f"generating user memory doc for the user : {name}")
                
                if 'history' not in memory.keys():
                    continue
                memory_arr = []
                for date, content in memory['history'].items():
                    memory_arr.append(f'Conversation on {date}：')
                    for dialog in content:
                        query = dialog['query']
                        response = dialog['response']
                        memory_arr.append(f'\n{user_name}：{query.strip()}')
                        memory_arr.append(f'\nAI：{response.strip()}')
                    memory_arr.append('\n')
                    if 'summary' in memory:
                        if date in memory['summary'].keys():
                            memory_arr.append(f'The summary of the conversation on {date} is: {memory["summary"][date]}')
                    if 'personality' in memory:
                        if date in memory['personality'].keys():
                            memory_arr.append(f'The personality analysis for the date {date} is: {memory["personality"][date]}')
                    memory_str = "".join(memory_arr)
                    # print("memory_str: ", memory_str)
                    user_memory_doc = Document(text=memory_str)
                    # print("user_memory_doc: ", user_memory_doc)
                    # print("type of user_memory_doc: ", type(user_memory_doc))
            return user_memory_doc

        
        # index_set = {}

        user_memory_doc = generate_user_memory_doc(user_name, memory_data)
        
        # llm_predictor = LLMPredictor(llm=OpenAIChat(model_name="gpt-3.5-turbo"))
        # prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)
        # service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
        

        cur_index = VectorStoreIndex.from_documents([user_memory_doc])
        # index_set[user_name] = cur_index
        os.makedirs(f'./memory_index/',exist_ok=True)
        # cur_index.save_to_disk(f'./memory_index/{user_name}_index.json')
        # cur_index.storage_context.persist(persist_dir=f'./memory_index/{user_name}_index.json')
        cur_index.storage_context.persist(persist_dir=f'./memory_index/{user_name}_index')
        
    
    # 1) update memory data
    update_memory_data(user_name, memory_data, memory_data_dir, input_text, response)
    
    # 2) update memory index
    update_memory_index(user_name, memory_data)



In [18]:
print('Please Enter Your Name')
user_name = input("\nUser Name：")

# Clear the history and start fresh
history = []

Please Enter Your Name



User Name： mimi


In [19]:
if user_name in memory_data:
    user_memory = memory_data[user_name]
    user_memory_index = get_user_memory_index(user_name)
    print(f"Welcome back {user_name}")
else: 
    print(f"Hello {user_name}, nice to meet you")

Hello mimi, nice to meet you


In [20]:
print(f"Welcome to use {boot_actual_name}，please enter your question to start conversation, enter \"stop\" to stop program :")
while True:
    input_text = input(f"\n{user_name}：")

    # exit the conversation
    if input_text.strip() == "stop":
                break

    # print("user_memory_index : ", user_memory_index)
    # print("all the methods: ", dir(user_memory_index))
    # step1: query the memory index if the user's memory_index exists
    related_memos = ""
    if user_memory_index:
        related_memos = get_related_memos_from_memory_index(user_memory_index, input_text)
    # print("rleated_memos : ", related_memos)
    
    # step2: build the prompt and request gpt api with related_info if it exists and prints the response
    result = ""
    updated_history_to_show_user, updated_history, response = build_prompt_and_ask_gpt(related_memos, user_memory, input_text, history)
    history = updated_history

    # step3: update the relevant piece of memory_data and memory_index on every interaction
    update_memory_data_and_memory_index(user_name, user_memory_index, memory_data, memory_data_dir, input_text, response)

Welcome to use LanguageFriend，please enter your question to start conversation, enter "stop" to stop program :



mimi： I love coding challenges


LanguageFriend : That's great to hear! Coding challenges can be a fun and engaging way to sharpen your programming skills. Is there a specific type of coding challenge you enjoy the most?
*** Updating memory for user mimi ***



mimi： stop
